第九章 符合python风格的对象

9.1 对象表示形式

每门面向对象的语言至少都有一种获取对象的字符串表示形式的标准形式。python提供了两种：

repr():以便于开发者理解的方式返回对象的字符串表示形式

str():以便于用户理解的方式返回对象的字符串表示方式

9.2 再谈向量类

In [1]:
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
# END VECTOR2D_V0

In [2]:
v1 = Vector2d(3,4)

In [3]:
print(v1.x,v1.y)

3.0 4.0


In [42]:
x,y=v1
type(x)

float

In [5]:
repr(v1)

'Vector2d(3.0, 4.0)'

In [6]:
eval(repr(v1))

Vector2d(3.0, 4.0)

In [4]:
v1_clone = eval(repr(v1))

In [7]:
v1==v1_clone

True

In [8]:
octets=bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [9]:
abs(v1)

5.0

In [10]:
bool(Vector2d(0,0))

False

In [11]:
array('d',Vector2d(3,4))

array('d', [3.0, 4.0])

In [12]:
bytes(_)

b'\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

9.3 备选构造方法

In [14]:
#上节向量类还缺一个吧二进制对象转换为向量的方法
#array.array有个类方法.frombytes正好符合需求
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
            return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    @classmethod
    def frombytes(cls,octets):
        typecode = chr(octets[0])
        memv=memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [19]:
Vector2d.frombytes(b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@')

Vector2d(3.0, 4.0)

9.4 classmethod与staticmethod

![chapter9-4](image/chapter9-4.png)

In [20]:
#比较classmethod和staticmethod的行为
class Demo:
    @classmethod
    def klassmeth(*args):#第一个参数是类本身
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [21]:
Demo.klassmeth()#第一个参数是类本身

(__main__.Demo,)

In [22]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [23]:
Demo.statmeth()

()

In [24]:
Demo.statmeth('spam')

('spam',)

9.5格式化显示

In [25]:
#内置的format()函数和str.format()方法把各个类型的格式化方式委托给相应的.__format__(format_spec)方法.format是格式说明符，它是：
#format(my_obj,format_spec)的第二个参数，或者
#str.format()方法的格式字符串，{}里代换字段中冒号后面的部分
brl = 1/2.43

In [26]:
brl

0.4115226337448559

In [27]:
format(brl,'0.4f')

'0.4115'

In [34]:
'1 BRL ={rate:10.2f} USD'.format(rate=brl)

'1 BRL =      0.41 USD'

In [35]:
format(42,'b')

'101010'

In [36]:
format(2/3,'.1%')

'66.7%'

In [37]:
#格式规范微语言是可扩展的,因为各个类可以自行决定如何解释format
from datetime import datetime
now=datetime.now()
format(now,'%H:%M:%S')

'13:13:59'

In [39]:
'It is now {:%I:%M %p}'.format(now)

'It is now 01:13 PM'

In [40]:
#如果类没有定义__format__方法，从object继承的方法会返回str(my_object)。
v1=Vector2d(3,4)
format(v1)

'(3.0, 4.0)'

In [41]:
#然而，如果传入格式说明符，object.__format__方法会抛出TypeError:
format(v1,'.3f')

TypeError: non-empty format string passed to object.__format__

In [43]:
#添加__format__方法
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    def __format__(self, format_spec=''):
        components=(format(c,format_spec) for c in self)
        return '({},{})'.format(*components)
# END VECTOR2D_V2

In [44]:
v1=Vector2d(3,4)
format(v1)

'(3.0,4.0)'

In [45]:
format(v1,'.2f')

'(3.00,4.00)'

In [46]:
format(v1,'.3e')

'(3.000e+00,4.000e+00)'

In [52]:
#下面要在微语言中添加一个自定义的格式代码：如果格式说明符以'p'结尾，那么在极坐标中显示向量
#添加__format__方法
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    def angle(self):
        return math.atan2(self.y,self.x)
    
    #重写format方法
    def __format__(self, format_spec=''):
        if format_spec.endswith('p'):
            format_spec=format_spec[:-1]
            coords=(abs(self),self.angle())
            outer_fmt = '<{} , {}>'
        else:
            coords=self
            outer_fmt='({} , {})'
        components=(format(c,format_spec) for c in coords)
        return outer_fmt.format(*components)
# END VECTOR2D_V2

In [53]:
format(Vector2d(1,1),'p')

'<1.4142135623730951 , 0.7853981633974483>'

In [54]:
format(Vector2d(1,1,),'.3ep')

'<1.414e+00 , 7.854e-01>'

下面，我们将把vector2d变成可散列的，这样便可以构成向量集合，或者把向量当做是dict的键使用。不过在此之前，必须让向量不可变

9.6 可散列的Vector2d

In [60]:
l1=[1,2,3]
id(l1)


1315033707720

In [61]:
l1+=[4,5]
id(l1)

1315033707720

In [62]:
hash(l1)

TypeError: unhashable type: 'list'

In [68]:
l2=(1,2,3)
id(l2)

1315033578664

In [69]:
hash(l2)

2528502973977326415

In [70]:
l3=(1,2,[1,2])
hash(l3)

TypeError: unhashable type: 'list'

In [71]:
#目前Vector2d实例是不可散列的，因此不能放入集合(set)中
v1=Vector2d(3,4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [72]:
#为了把Vector2d实例变成可散列的，必须使用__hash__方法（还需要__eq__方法，前面已经实现了）。此外，还需让向量不可变
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.__x = float(x)    # <2>
        self.__y = float(y)#把属性标记为私有的
        
    @property #把读值方法标记为特性
    def x(self):#读值方法与公开属性同名，都是x
        return self.__x
    
    @property #把读值方法标记为特性
    def y(self):#读值方法与公开属性同名，都是y
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))  # 需要读取x和y分量的方法可以保持不变，通过self.x和self.y读取公开特性，
                                              # 而不必读取私有属性

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    def angle(self):
        return math.atan2(self.y,self.x)
    
    #重写format方法
    def __format__(self, format_spec=''):
        if format_spec.endswith('p'):
            format_spec=format_spec[:-1]
            coords=(abs(self),self.angle())
            outer_fmt = '<{} , {}>'
        else:
            coords=self
            outer_fmt='({} , {})'
        components=(format(c,format_spec) for c in coords)
        return outer_fmt.format(*components)
    
    #d定义哈希方法
    def __hash__(self):
        return hash(self.x) ^ hash(self.y) #最后使用位运算异或混合各分量的散列值
    
# END VECTOR2D_V3

In [73]:
v1=Vector2d(3,4)

In [74]:
v2=Vector2d(3.1,4.2)

In [75]:
hash(v1),hash(v2)

(7, 384307168202284039)

In [76]:
set([v1,v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

9.7 python的私有属性和“受保护的”属性

![chapter9-7](image/chapter9-7.png)

In [77]:
v1=Vector2d(3,4)

In [78]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [80]:
v1._Vector2d__x=6

In [81]:
v1

Vector2d(6, 4.0)

In [82]:
hash(v1)

2

In [83]:
#为了把Vector2d实例变成可散列的，必须使用__hash__方法（还需要__eq__方法，前面已经实现了）。此外，还需让向量不可变
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self._x = float(x)    # <2>
        self._y = float(y)#把属性标记为私有的
        
    @property #把读值方法标记为特性
    def x(self):#读值方法与公开属性同名，都是x
        return self._x
    
    @property #把读值方法标记为特性
    def y(self):#读值方法与公开属性同名，都是y
        return self._y

    def __iter__(self):
        return (i for i in (self.x, self.y))  # 需要读取x和y分量的方法可以保持不变，通过self.x和self.y读取公开特性，
                                              # 而不必读取私有属性

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6> self.typecode没有定义，访问的是类属性
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    def angle(self):
        return math.atan2(self.y,self.x)
    
    #重写format方法
    def __format__(self, format_spec=''):
        if format_spec.endswith('p'):
            format_spec=format_spec[:-1]
            coords=(abs(self),self.angle())
            outer_fmt = '<{} , {}>'
        else:
            coords=self
            outer_fmt='({} , {})'
        components=(format(c,format_spec) for c in coords)
        return outer_fmt.format(*components)
    
    #d定义哈希方法
    def __hash__(self):
        return hash(self.x) ^ hash(self.y) #最后使用位运算异或混合各分量的散列值
    
# END VECTOR2D_V3

In [84]:
v1=Vector2d(3,4)
v1.__dict__

{'_x': 3.0, '_y': 4.0}

In [85]:
v1._x=6

In [86]:
v1

Vector2d(6, 4.0)

总之，Vector2d的分量都是“私有的”的，而且Vector2d实例都是“不可变”的，用引号是因为并不能真正实现私有和不可变

9.8 使用__slots__类属性节省空间

![chapter9-8](image/chapter9-8.png)

In [87]:

from array import array
import math


class Vector2d:
    
    __slots__ = ('__x','__y')#这样，python会在各个实例中使用类似元祖的结构存储实例变量，从而避免使用消耗内存的__dict__属性。
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.__x = float(x)    # <2>
        self.__y = float(y)#把属性标记为私有的
        
    @property #把读值方法标记为特性
    def x(self):#读值方法与公开属性同名，都是x
        return self.__x
    
    @property #把读值方法标记为特性
    def y(self):#读值方法与公开属性同名，都是y
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))  # 需要读取x和y分量的方法可以保持不变，通过self.x和self.y读取公开特性，
                                              # 而不必读取私有属性

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    def angle(self):
        return math.atan2(self.y,self.x)
    
    #重写format方法
    def __format__(self, format_spec=''):
        if format_spec.endswith('p'):
            format_spec=format_spec[:-1]
            coords=(abs(self),self.angle())
            outer_fmt = '<{} , {}>'
        else:
            coords=self
            outer_fmt='({} , {})'
        components=(format(c,format_spec) for c in coords)
        return outer_fmt.format(*components)
    
    #d定义哈希方法
    def __hash__(self):
        return hash(self.x) ^ hash(self.y) #最后使用位运算异或混合各分量的散列值
    
# END VECTOR2D_V3

In [88]:
v1=Vector2d(3,4)

In [89]:
v1.__dict__

AttributeError: 'Vector2d' object has no attribute '__dict__'

In [93]:
v1.__slots__

('__x', '__y')

9.9 覆盖类属性

![chapter9-9](image/chapter9-9.jpg)

In [95]:
v1=Vector2d(1.1,2.2)
dumpd=bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [96]:
len(dumpd)

17

In [97]:
v1.typecode='f'
dumpf=bytes(v1)

AttributeError: 'Vector2d' object attribute 'typecode' is read-only

In [98]:
class Per:
    name='per'
p=Per()

In [99]:
p.name

'per'

In [100]:
p.name='aaa'

In [101]:
p.name

'aaa'

In [102]:
Per.name

'per'

In [103]:
class ShortVector2d(Vector2d):
    typecode='f'
sv=ShortVector2d(1/11,1/27)
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [105]:
len(bytes(sv))

9